In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# 차트 내 한글 및 마이너스 기호 표시를 위한 import
from matplotlib import rc
import matplotlib.font_manager as fm

import matplotlib.pyplot as plt

plt.rc('font', family='Malgun Gothic')
sns.set(font='Malgun Gothic', rc={'axes.unicode_minus':False},style='darkgrid')


from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv("./(최종)통합데이터.csv",encoding='cp949')
df.head()

,역사명,동별,위도,경도,일평균승하차(명),상가 면적,공실면적(㎡)(2024년기준),거주자수,남여비율,1인 가구수 레벨,...,20~40대 인구 수 레벨,고속버스 터미널,기차,여객 터미널,주변 관광지,주변 대학교,일평균(원),kmeans_group,gmm_group,kmedoids
0,가야,가야1동,35.155883,129.042817,5799,11.0,11.0,25217,0.95,83,...,57,NaN,1.0,NaN,0,0,1926082,0,0,3
1,감전,감전동,35.155528,128.991146,6107,10.0,10.0,12825,1.12,56,...,31,NaN,NaN,NaN,0,0,2535118,0,0,3
2,강서구청,대저1동,35.211247,128.981756,4251,NaN,NaN,2076,1.22,26,...,4,NaN,NaN,NaN,0,0,2029127,0,0,3
3,개금,개금1동,35.153284,129.020533,13621,37.0,0.0,27187,0.94,79,...,65,NaN,NaN,NaN,0,1,4717075,3,0,1
4,거제(법원·검찰청),거제3동,35.188589,129.073941,9546,16.0,0.0,27403,0.91,14,...,13,NaN,NaN,NaN,0,1,3099526,0,0,3


In [3]:
df.columns

Index(['역사명', '역위도', '역경도', '일평균승하차(명)', '상가 면적', '거주자수', '남여비율', '1인 가구수 레벨',
       '1인 가구수(세대)', '사업체수', '20~40대 인구 수 레벨', '고속버스 터미널', '기차역', '여객 터미널',
       '주변 관광지', '주변 대학교', '일평균(원)'],
      dtype='object')

In [3]:
use_df = df[['역사명','일평균승하차(명)','1인 가구수(세대)','사업체수','일평균(원)']]
use_df.head()

,역사명,일평균승하차(명),1인 가구수(세대),사업체수,일평균(원)
0,가야,5799,6291,49,1926082
1,감전,6107,5461,57,2535118
2,강서구청,4251,4630,109,2029127
3,개금,13621,2501,36,4717075
4,거제(법원·검찰청),9546,3093,114,3099526


In [4]:
use_df.isnull().sum()

역사명           0
일평균승하차(명)     0
1인 가구수(세대)    0
사업체수          0
일평균(원)        0
dtype: int64

In [5]:
use_df = use_df.set_index(use_df["역사명"])

In [6]:
use_df.drop(columns="역사명", inplace=True)

In [7]:
use_df

,일평균승하차(명),1인 가구수(세대),사업체수,일평균(원)
역사명,,,,
가야,5799,6291,49,1926082
감전,6107,5461,57,2535118
강서구청,4251,4630,109,2029127
개금,13621,2501,36,4717075
거제(법원·검찰청),9546,3093,114,3099526
...,...,...,...,...
충렬사(안락),5398,1851,35,1117126
토성,13846,2110,7,6002230
하단,29080,4414,36,11010043


# k-medoids 알고리즘을 이용한 클러스터링

In [ ]:
# !pip install scikit-learn-extra

In [ ]:
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=4, max_iter=30,random_state=42).fit(use_df)
kmedoids.labels_

In [ ]:
# 시각화
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

plt.figure(figsize=(20,15))
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
pca.fit(use_df)
pca_data = pca.transform(use_df)
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=kmedoids.labels_, alpha=0.5)
plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)

pca에서 위와 같은 코드로 간단하게 내가 설정한 주성분의 개수(n_components)로 전체 데이터의 분산을 얼마만큼 설명 가능한지 알 수 있다.\
본 데이터의 경우 두 개의 주성분이 전체 분산의 약 99%를 설명한다.